In [1]:
import io
from PIL import Image
import os
import wandb
import glob
import torch
import monai
import random
import numpy as np
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import matplotlib.pyplot as plt
import segmentation_models_pytorch as smp

In [2]:
torch.manual_seed(1024)
np.random.seed(1024)
device = torch.device(
    "cuda:1"
    if torch.cuda.is_available()
    else
    "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# Data Augmentation
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize([0.45, ], [0.35, ]),
    transforms.Resize([256, 256]),
    transforms.RandomRotation(45),
    transforms.RandomResizedCrop([256, 256])
])

target_transform = transforms.Compose([
    transforms.Resize([256, 256], interpolation=transforms.InterpolationMode.NEAREST),
    transforms.RandomRotation(45, interpolation=transforms.InterpolationMode.NEAREST),
    transforms.RandomResizedCrop([256, 256], interpolation=transforms.InterpolationMode.NEAREST)
])

# Validation Data Augmentation
val_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize([0.45, ], [0.35, ]),
    transforms.Resize([256, 256])
])

val_target_transform = transforms.Compose([
    transforms.Resize([256, 256], interpolation=transforms.InterpolationMode.NEAREST)
])

Using cuda:1 device


In [3]:
lr = 6e-5
batch_size = 8
weight_decay = 1e-5
num_epochs = 20

run = wandb.init(
    project="Unet-Final",
    name='Unet-Resnet50',
    # Track hyperparameters and run metadata
    config={
        "learning rate": lr,
        "batch_size": batch_size,
        "weight decay": weight_decay,
        "Epoches number": num_epochs,
        "transform": str(transform),
        "target transform": str(target_transform)
    })

2023-06-16 12:27:08,626 - Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Currently logged in as: ming686 (deeplearning-med). Use `wandb login --relogin` to force relogin


In [4]:
class SegDataset(Dataset):
    def __init__(self, data_root, transform, target_transform, 
                 include_background = True, train=True, to3d=False):
        self.data_root = data_root
        self.transform = transform
        self.target_transform = target_transform
        self.train = train
        self.to3d = to3d
        self.gt_files_path = []
        self.include_background = include_background
        
        # find all patient directories
        patient_directories = glob.glob(os.path.join(self.data_root, 'patient*'))
        # find all files with the suffix _gt.npy
        train_size = int(len(patient_directories)*0.8)
        
        if self.train:
            for patient_directory in patient_directories[0:train_size]:
                per_patient_file_path = glob.glob(os.path.join(patient_directory, '*_gt.npy'))
                for path in per_patient_file_path:
                    self.gt_files_path.append(path)
        else:
            for patient_directory in patient_directories[train_size:]:
                per_patient_file_path = glob.glob(os.path.join(patient_directory, '*_gt.npy'))
                for path in per_patient_file_path:
                    self.gt_files_path.append(path)
                    
    def __len__(self):
        return len(self.gt_files_path)
    
    def __getitem__(self, index):
        gt_image_path = self.gt_files_path[index]
        image_path = gt_image_path[:-7] + ".npy"
        image = np.load(image_path)
        gt_image = np.load(gt_image_path)
        image = torch.tensor(image[None,:,:]).float()
        gt_image = torch.tensor(gt_image).long()
        # Convert the ground truth label to one-hot encoding
        one_hot_label = torch.nn.functional.one_hot(gt_image, num_classes=4)

        # Transpose the tensor to have dimensions (C, H, W)
        one_hot_label = one_hot_label.permute(2, 0, 1)
        
        if not self.include_background:
            # Remove the background channel (dimension 0)
            one_hot_label = one_hot_label[1:, :, :]
        
        # Use seed to make sure image and target has same transform
        seed = np.random.randint(2147483647)
        random.seed(seed)
        torch.manual_seed(seed)
        image = self.transform(image)

        random.seed(seed)
        torch.manual_seed(seed)
        target = self.target_transform(one_hot_label)
        
        # For 3 dimension input channel
        if self.to3d:
            rgb_tensor = image.repeat(3, 1, 1)
            return rgb_tensor, target
        return image, target

In [5]:
train_dataset = SegDataset(data_root = './database/training', 
                     transform = transform, 
                     target_transform = target_transform,
                     include_background = True,
                     train = True,
                     to3d = False)

val_dataset = SegDataset(data_root = './database/training', 
                     transform = val_transform, 
                     target_transform = val_target_transform,
                     include_background = True,
                     train = False,
                     to3d = False)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

In [6]:
model = smp.Unet(
    encoder_name="resnet50",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=1,                  # model input channels
    classes=4                       # model output channels (number of classes)
)

# preprocess_input = get_preprocessing_fn('mit_b3', pretrained='imagenet')

model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
seg_loss = monai.losses.TverskyLoss(sigmoid=True)

In [7]:
def vis_img(img, mask):
    # img: (B, 256, 64, 64), {: (B, 1, 256, 256)
    #print(f"{img.shape=}, {mask.shape=}")
    img = np.squeeze(img)
    mask = np.squeeze(mask)
    plt.figure()  
    plt.imshow(img, 'gray')
    overlay_mask_1 = np.ma.masked_where(mask[1] == 0, img)
    overlay_mask_2 = np.ma.masked_where(mask[2] == 0, img)
    overlay_mask_3 = np.ma.masked_where(mask[3] == 0, img)
    plt.imshow(overlay_mask_1, 'Greens', alpha = 1, interpolation='nearest')
    plt.imshow(overlay_mask_2, 'Reds', alpha = 1, interpolation='nearest')
    plt.imshow(overlay_mask_3, 'Purples', alpha = 1, interpolation='nearest')
    buffer = io.BytesIO()
    plt.savefig(buffer, format='jpeg')
    buffer.seek(0)

    # Convert the in-memory buffer to a NumPy array
    image_array = np.array(Image.open(buffer))
    
    plt.close()
    
    return image_array

In [8]:
# train
best_loss = 1e10
val_freq = 2

for epoch in range(num_epochs):
    # Train
    model.train()
    epoch_loss = 0
    for step, (img, gt) in enumerate(tqdm(train_loader)):
        img = img.to(device)
        mask = model(img)
        loss = seg_loss(mask, gt.to(device))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    
    print(f'EPOCH: {epoch + 1}, Train Loss: {epoch_loss}')
    
    # Validation
    if epoch % val_freq == 0:
        model.eval()
        val_loss = 0
        last_image_batch = None
        last_gt_mask_batch = None
        last_pr_mask_batch = None
        with torch.no_grad():
            for step, (img, gt) in enumerate(tqdm(val_loader)):
                img = img.to(device)
                mask = model(img)
                loss = seg_loss(mask, gt.to(device))
                val_loss += loss.item()
                last_image_batch = img
                last_gt_mask_batch = gt
                last_pr_mask_batch = mask
            
    print(f'EPOCH: {epoch + 1}, Validation Loss: {val_loss}')
    
    last_image = last_image_batch.detach().cpu().numpy()[0][0]
    last_gt = last_gt_mask_batch.detach().cpu().numpy()[0]
    last_pr = last_pr_mask_batch.detach().cpu().numpy()[0]
    
    threshold = 0.95  # Set your desired threshold value
    binary_mask = (last_pr > threshold)
    
    ground_truth = vis_img(last_image, last_gt)
    predicted = vis_img(last_image, binary_mask)
    # Log
    wandb.log({"loss": epoch_loss,
               "val_loss": val_loss,
               "ground_truth": wandb.Image(ground_truth),
               "prediction": wandb.Image(predicted)})
    
    # save the best model
    if epoch_loss < best_loss:
        best_loss = epoch_loss
        torch.save(model.state_dict(), './model/Final/Unet-res50/model_best.pth')
torch.save(model.state_dict(), './model/Final/Unet-res50/model_final_epoch.pth')

  3%|▎         | 6/192 [00:06<03:30,  1.13s/it]


KeyboardInterrupt: 